In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import warnings

from matplotlib import rc

import mplcyberpunk
plt.style.use('cyberpunk')
warnings.filterwarnings(action="ignore")

rc('font', family='AppleGothic') 			
plt.rcParams['axes.unicode_minus'] = False
mpl.rc('font', size=12)
%matplotlib inline

In [5]:
data_path = '/Users/tastebread/Desktop/kaggle/competitive-data-science-predict-future-sales/'
sales_train = pd.read_csv(data_path + 'sales_train.csv')
test = pd.read_csv(data_path + 'test.csv')
sample_submission = pd.read_csv(data_path + 'sample_submission.csv')
item_categories = pd.read_csv(data_path + 'item_categories.csv')
items = pd.read_csv(data_path + 'items.csv')
shops = pd.read_csv(data_path + 'shops.csv')

In [6]:
display(sales_train.head())
display(test.head())
display(sample_submission.head())
display(item_categories.head())
display(items.head())
display(shops.head())

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [9]:
"""
피처 엔지니어링I
이번 데이터셋은 여러 파일, 피쳐도 다양하다. 피쳐명이 영어면 조금 헷갈릴 수 있기 때문에 다양한 파생 피쳐를 만들어볼 계획이라면 쉽게 알아볼수 있도록
피쳐명을 한글로 바꿔즈는것이 좋다
"""

sales_train = sales_train.rename(columns={'date':'날짜',
                                          'date_block_num':'월ID',
                                          'shop_id':'상점ID',
                                          'item_id':'상품ID',
                                          'item_price':'판매가',
                                          'item_cnt_day':'판매량'})

display(sales_train.head(5))

shops = shops.rename(columns={'shop_name':'상점명',
                              'shop_id':'상점ID'})

display(shops.head(5))

items = items.rename(columns={'item_name':'상품명',
                              'item_id':'상품ID',
                              'item_category_id':'상품분류ID'})

display(items.head(5))

item_categories = item_categories.rename(columns={'item_category_name':'상품분류명',
                                                  'item_category_id':'상품분류ID'})

display(item_categories.head(5))

test = test.rename(columns={'shop_id':'상점ID',
                            'item_id':'상품ID'})

display(test.head(5))

,날짜,월ID,상점ID,상품ID,판매가,판매량
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


,상점명,상점ID
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


,상품명,상품ID,상품분류ID
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


,상품분류명,상품분류ID
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


,ID,상점ID,상품ID
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [11]:
"""
피쳐 엔지니어링 II
테스트 데이터의 피처는 ID를 제외하면 상점ID,상품ID 피처이고, 우리가 예측하는 값은 각 상점의 상품별 월간 판매량이었다.
월간 판매량을 예측하려면 월,상점(상점ID),상품별(상품ID) 조합이 필요하다 여기서 조합한다는것은
원본 데이터에 없는 상품등을 만들어서 의미 있는 데이터로 만들어 가는것
"""
from itertools import product # 여러 Iterable 요소들의 순서쌍을 반환하는 Iterator 생성

train = []

#월ID, 상점ID, 상품ID 조합을 생성
for i in sales_train['월ID'].unique():
    all_shop = sales_train.loc[sales_train['월ID'] == i, '상점ID'].unique()
    all_item = sales_train.loc[sales_train['월ID'] == i, '상품ID'].unique()
    train.append(np.array(list(product([i], all_shop,all_item))))

idx_features = ['월ID','상점ID','상품ID'] #기준이 되는피쳐 생성

#리스트 타입을 DataFrame으로 변환
train = pd.DataFrame(np.vstack(train), columns=idx_features)

train #원본 데이터 개수보다 약 3배가량정도 더 늘었다. 이 train을 뼈대로 타깃값을 병합하고 나머지 데이터셋도 병합해준다.


,월ID,상점ID,상품ID
0,0,59,22154
1,0,59,2552
2,0,59,2554
3,0,59,2555
4,0,59,2564
...,...,...,...
10913845,33,21,7635
10913846,33,21,7638
10913847,33,21,7640
10913848,33,21,7632


In [17]:
"""
피처 엔지니어링III : 타깃값(월간 판매량 추가)
타깃값인 각 상점의 상품별 월간 판매량 피처 추가하기.

월간 판매량을 구하려면 월ID,상점ID,상품ID를 기준으로 그룹화하고 판매량을 더해야한다.
groupby() 함수를 활용하여 월간 판매량을 구한다.
"""
#idx_features 기준으로 그룹화
group = sales_train.groupby(idx_features).agg({'판매량':'sum'})

#인덱스 재설정
group.reset_index()

group = group.rename(columns={'판매량':'월간 판매량'})

display(group)

#train = train.merge(group, on=idx_features,how='left')
#train.drop(['월간 판매량_x','월간 판매량_y'],axis=1,inplace=True)
display(train)

#월간 판매량에 NaN값이 많다. 그 이유는 기존에 없던 조합에는 판매량 정보가 없기때문
#판매량 정보가 없다면 0이라는 뜻이니 결측값은 0으로 대체

월간 판매량
월ID 상점ID 상품ID         
0   0    32        6.0
         33        3.0
         35        1.0
         43        1.0
         51        2.0
...                ...
33  59   22087     6.0
         22088     2.0
         22091     1.0
         22100     1.0
         22102     1.0

[1609124 rows x 1 columns]

,월ID,상점ID,상품ID,월간 판매량
0,0,59,22154,1.0
1,0,59,2552,NaN
2,0,59,2554,NaN
3,0,59,2555,NaN
4,0,59,2564,NaN
...,...,...,...,...
10913845,33,21,7635,NaN
10913846,33,21,7638,NaN
10913847,33,21,7640,NaN
10913848,33,21,7632,NaN


In [18]:
"""
피처 엔지니어링IV 
데이터셋을 병합 과정 이때 테스트 데이터도 병합해주면 좋기에 테스트 데이터부터 병합해준다

병합하기 전에 test 데이터는 월ID가 없으므로 월ID를 추가해줘야한다.

월 ID 0 은 2013년 1월이고, 33은 2015년 10월이다
테스트 데이터는 2015년 11월 판매기록이기때문에 테스트 데이터의 월ID는 34로 설정해준다.
"""
test['월ID'] = 34

In [21]:
#train 과 test 이어붙이기

df = pd.concat([train,test.drop('ID',axis=1)],
               ignore_index=True, #기존 인덱스 무시(0부터 새로 시작)
               keys=idx_features)  #이어붙이는 기준이 되는 피쳐

In [23]:
df = df.fillna(0) #NaN값 0으로 대체

df

,월ID,상점ID,상품ID,월간 판매량
0,0,59,22154,1.0
1,0,59,2552,0.0
2,0,59,2554,0.0
3,0,59,2555,0.0
4,0,59,2564,0.0
...,...,...,...,...
11128045,34,45,18454,0.0
11128046,34,45,16188,0.0
11128047,34,45,15757,0.0
11128048,34,45,19648,0.0


In [24]:
"""
피처 엔지니어링 V 
나머지 데이터 병합(최종 데이터 생성)
shops,items,item_categories 데이터를 df 데이터에 병합하기
"""

df = df.merge(shops, on='상점ID',how='left')
df = df.merge(items, on='상품ID',how='left')
df = df.merge(item_categories, on='상품분류ID',how='left')

df.head(5)

,월ID,상점ID,상품ID,월간 판매량,상점명,상품명,상품분류ID,상품분류명
0,0,59,22154,1.0,"Ярославль ТЦ ""Альтаир""",ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray
1,0,59,2552,0.0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
2,0,59,2554,0.0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил
3,0,59,2555,0.0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства
4,0,59,2564,0.0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE Perihelion: Live In Concert DVD (К...,59,Музыка - Музыкальное видео


In [25]:
#상점명, 상품명, 상품분류명은 문자데이터지만 각ID피쳐들과 일대일로 매칭되므로 모두 제거

df = df.drop(['상점명','상품명','상품분류명'],axis=1)

In [26]:
import gc
"""
훈련,검증,테스트용 데이터 제작.

훈련 데이터 : 2013년 1월부터 2015년 9월(월ID=32)까지의 판매내역
검증 데이터 : 2015년 10월(월ID=33) 판매 내역
테스트 데이터 : 2015년 11월(월ID=34) 판매 내역
"""

#훈련 데이터
X_train = df[df['월ID'] < 33]
X_train = X_train.drop(['월간 판매량'], axis=1)

#검증 데이터
X_val = df[df['월ID'] == 33]
X_val = X_val.drop(['월간 판매량'],axis=1)

#테스트 데이터
X_test = df[df['월ID'] == 34]
X_test = X_test.drop(['월간 판매량'],axis=1)

#훈련 데이터 (타깃값)
y_train = df[df['월ID'] < 33]['월간 판매량']

#검증 데이터 (타깃값)
y_val = df[df['월ID'] == 33]['월간 판매량']

#훈련,검증,테스트 데이터를 할당했으니 필요없는 df 는 제거

del df
gc.collect();

In [31]:
#모델 훈련 및 성능 검증으로 LightGBM 사용

import lightgbm as lgb

#lightgbm 용 하이퍼 파라미터

params = {'metric':'rmse', #평가지표는 rmse
          'num_leaves':255,
          'learning_rate':0.01,
          'force_col_wise':'True',
          'random_state':' 10'}

# 범주형 피쳐 설정
cat_feature = ['상점ID','상품분류ID']

#LGB용 훈련 및 검증 데이터셋
dtrain = lgb.Dataset(X_train,y_train)
dval = lgb.Dataset(X_val,y_val)

lgb_ml = lgb.train(params=params,
                   train_set=dtrain,
                   num_boost_round=500,
                   valid_sets=(dtrain,dval),
                   categorical_feature=cat_feature)

TypeError: train() got an unexpected keyword argument 'verbose_eval'